# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])
               
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import pandas as pd
from sqlalchemy import create_engine
import pickle

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/sibae/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sibae/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/Database.db')
df = pd.read_sql_table('disaster_details', engine)
X = df['message']
Y = df.drop(columns=['id', 'original', 'genre', 'message']).values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
        
    return clean_tokens
        

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer= tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=25)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x286e083a0>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=25)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
pred = pipeline.predict(X_test)

In [7]:
for i in range(pred.shape[1]):
    print("*******" + df.columns[4+i] + "*******")
    print(classification_report(y_test[:, i], pred[:, i]))
    print("\n")

*******related*******
              precision    recall  f1-score   support

           0       0.74      0.27      0.40      1558
           1       0.80      0.97      0.88      4951
           2       0.50      0.20      0.29        45

    accuracy                           0.80      6554
   macro avg       0.68      0.48      0.52      6554
weighted avg       0.79      0.80      0.76      6554



*******request*******
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5449
           1       0.87      0.45      0.59      1105

    accuracy                           0.90      6554
   macro avg       0.89      0.72      0.76      6554
weighted avg       0.89      0.90      0.88      6554



*******offer*******
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   mac

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(tokenizer=<function tokenize at 0x286e083a0>)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(random_state=25)))],
 'verbose': False,
 'tfidf': TfidfVectorizer(tokenizer=<function tokenize at 0x286e083a0>),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(random_state=25)),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': None,
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': <function __main__.tokenize(text)>,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None

In [15]:
parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__estimator__n_estimators': (5 ,10)
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1)

In [16]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x286e083a0>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(random_state=25)))]),
             param_grid={'clf__estimator__n_estimators': (5, 10),
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             verbose=1)

In [17]:
category_names = df.drop(columns=['id', 'original', 'genre', 'message']).columns.tolist()
Y_pred_cv = cv.predict(X_test)
Y_pred_cv_df = pd.DataFrame(Y_pred_cv, columns = category_names)
Y_pred_cv_df

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6550,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6551,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6552,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
for i in range(len(category_names)):
    print("*******--" + category_names[i] + "--*******")
    print(classification_report(y_test[:, i], Y_pred_cv_df.iloc[:, i]))
    print("\n")

*******--related--*******
              precision    recall  f1-score   support

           0       0.61      0.35      0.44      1558
           1       0.82      0.93      0.87      4951
           2       0.47      0.20      0.28        45

    accuracy                           0.79      6554
   macro avg       0.63      0.49      0.53      6554
weighted avg       0.76      0.79      0.76      6554



*******--request--*******
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5449
           1       0.83      0.44      0.57      1105

    accuracy                           0.89      6554
   macro avg       0.86      0.71      0.75      6554
weighted avg       0.88      0.89      0.88      6554



*******--offer--*******
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
tfidf = TfidfVectorizer(tokenizer= tokenize).fit_transform(X_test)

In [ ]:
tfidf

### 9. Export your model as a pickle file

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.